# Bible Word Analyser

## How to use

Enter a strongs number to the text box, and run all cells

In [2]:
strongs_number = input("Enter a Strong's number: ")
print ("You have selected: " + strongs_number)

Enter a Strong's number: 334
You have selected: 334
